In [2]:
class Noeud:
    def __init__(self, grille=None, joueur_actuel=1, phase='placement'):
        self.grille = grille if grille else [0] * 9  # 0: vide, 1: joueur 1, 2: joueur 2
        self.joueur_actuel = joueur_actuel
        self.phase = phase  # 'placement' ou 'déplacement'

    def est_victoire(self, joueur):
        alignements_gagnants = [
            (0, 1, 2), (3, 4, 5), (6, 7, 8),  # Lignes
            (0, 3, 6), (1, 4, 7), (2, 5, 8),  # Colonnes
            (0, 4, 8), (2, 4, 6)              # Diagonales
        ]
        return any(all(self.grille[i] == joueur for i in alignement) for alignement in alignements_gagnants)

    def joueur_a_gagne(self):
        # Ici, on considère que c'est le joueur qui vient de jouer qui a gagné.
        return self.est_victoire(3 - self.joueur_actuel)

    def nombre_de_pions(self, joueur):
        return sum(1 for x in self.grille if x == joueur)

    def coups_possibles(self):
        if self.phase == 'placement':
            return [i for i in range(9) if self.grille[i] == 0]
        else:
            # Phase de déplacement : on génère des coups sous forme de tuple (origine, destination)
            moves = []
            for i in range(9):
                if self.grille[i] == self.joueur_actuel:
                    for j in range(9):
                        if self.grille[j] == 0:
                            # Vous pouvez ajouter ici une condition pour limiter aux cases adjacentes
                            moves.append((i, j))
            return moves

    def jouer_coup(self, coup):
        nouvelle_grille = self.grille[:]
        if self.phase == 'placement':
            if nouvelle_grille[coup] == 0:
                nouvelle_grille[coup] = self.joueur_actuel
                # Passage à la phase déplacement si chaque joueur a 3 pions
                nouvelle_phase = self.phase
                if self.nombre_de_pions(1) >= 3 and self.nombre_de_pions(2) >= 3:
                    nouvelle_phase = 'déplacement'
                return Noeud(nouvelle_grille, 3 - self.joueur_actuel, nouvelle_phase)
        else:
            origine, destination = coup
            if nouvelle_grille[origine] == self.joueur_actuel and nouvelle_grille[destination] == 0:
                nouvelle_grille[origine] = 0
                nouvelle_grille[destination] = self.joueur_actuel
                return Noeud(nouvelle_grille, 3 - self.joueur_actuel, self.phase)
        return None

    def get_successor(self):
        return [self.jouer_coup(coup) for coup in self.coups_possibles() if self.jouer_coup(coup)]

    def evaluer(self):
        if self.est_victoire(1):
            return 1
        elif self.est_victoire(2):
            return -1
        return 0

    def minimax(self, profondeur, alpha, beta, maximiser):
        if profondeur == 0 or self.joueur_a_gagne():
            return self.evaluer()

        if maximiser:
            max_val = -float('inf')
            for successeur in self.get_successor():
                max_val = max(max_val, successeur.minimax(profondeur - 1, alpha, beta, False))
                alpha = max(alpha, max_val)
                if beta <= alpha:
                    break
            return max_val
        else:
            min_val = float('inf')
            for successeur in self.get_successor():
                min_val = min(min_val, successeur.minimax(profondeur - 1, alpha, beta, True))
                beta = min(beta, min_val)
                if beta <= alpha:
                    break
            return min_val

    def meilleur_coup(self, profondeur):
        best_move = None
        best_value = -float('inf')
        for coup in self.coups_possibles():
            successeur = self.jouer_coup(coup)
            move_value = successeur.minimax(profondeur, -float('inf'), float('inf'), False)
            if move_value > best_value:
                best_value = move_value
                best_move = coup
        return best_move

    def afficher(self):
        symboles = {0: '.', 1: 'X', 2: 'O'}
        for i in range(0, 9, 3):
            print(" ".join(symboles[self.grille[i + j]] for j in range(3)))
        print()

# Exemple d'utilisation de l'IA
def jeu_contre_ia():
    noeud = Noeud(joueur_actuel=1)  # Phase de placement initiale
    while not noeud.joueur_a_gagne():
        noeud.afficher()
        if noeud.joueur_actuel == 1:  # IA
            print("L'IA joue :")
            coup = noeud.meilleur_coup(profondeur=3)
            noeud = noeud.jouer_coup(coup)
            print(f"L'IA a joué : {coup}")
        else:  # Joueur humain
            if noeud.phase == 'placement':
                coup = int(input("Entrez une position (0-8) pour placer votre pion : "))
            else:
                origine = int(input("Choisissez le pion à déplacer (position 0-8) : "))
                destination = int(input("Choisissez la case d'arrivée (position 0-8) : "))
                coup = (origine, destination)
            nouveau_noeud = noeud.jouer_coup(coup)
            if nouveau_noeud is not None:
                noeud = nouveau_noeud
            else:
                print("Coup invalide, réessayez.")
    noeud.afficher()
    print(f"Le joueur {3 - noeud.joueur_actuel} a gagné !")

# Lancer le jeu
jeu_contre_ia()


. . .
. . .
. . .

L'IA joue :
L'IA a joué : 0
X . .
. . .
. . .

Entrez une position (0-8) pour placer votre pion : 2
X . O
. . .
. . .

L'IA joue :
L'IA a joué : 1
X X O
. . .
. . .

Entrez une position (0-8) pour placer votre pion : 3
X X O
O . .
. . .

L'IA joue :
L'IA a joué : 4
X X O
O X .
. . .

Entrez une position (0-8) pour placer votre pion : 8
X X O
O X .
. . O

L'IA joue :
L'IA a joué : 5
X X O
O X X
. . O

Choisissez le pion à déplacer (position 0-8) : 1
Choisissez la case d'arrivée (position 0-8) : 6
Coup invalide, réessayez.
X X O
O X X
. . O

Choisissez le pion à déplacer (position 0-8) : 8
Choisissez la case d'arrivée (position 0-8) : 7
X X O
O X X
. O .

L'IA joue :
L'IA a joué : (0, 6)
. X O
O X X
X O .



KeyboardInterrupt: Interrupted by user

In [6]:
class Noeud:
    def __init__(self, grille=None, joueur_actuel=1, phase='placement'):
        # La grille est une liste de 9 cases (0: vide, 1: joueur 1, 2: joueur 2)
        self.grille = grille[:] if grille else [0] * 9
        self.joueur_actuel = joueur_actuel
        self.phase = phase  # 'placement' ou 'déplacement'

    def est_victoire(self, joueur):
        """Vérifie si le joueur 'joueur' a réalisé un alignement gagnant."""
        alignements_gagnants = [
            (0, 1, 2), (3, 4, 5), (6, 7, 8),  # Lignes
            (0, 3, 6), (1, 4, 7), (2, 5, 8),  # Colonnes
            (0, 4, 8), (2, 4, 6)              # Diagonales
        ]
        return any(all(self.grille[i] == joueur for i in alignement) for alignement in alignements_gagnants)

    def joueur_a_gagne(self):
        """
        On considère que c'est le joueur qui vient de jouer (donc 3 - joueur_actuel)
        qui pourrait avoir réalisé un alignement gagnant.
        """
        return self.est_victoire(3 - self.joueur_actuel)

    def nombre_de_pions(self, joueur):
        """Retourne le nombre de pions du joueur sur le plateau."""
        return self.grille.count(joueur)

    def coups_possibles(self):
        """
        Renvoie la liste des coups possibles.
        - En phase de placement : la liste des cases vides (indices 0 à 8).
        - En phase de déplacement : la liste des tuples (origine, destination)
          où la case 'origine' contient un pion du joueur courant et 'destination' est vide.
        """
        if self.phase == 'placement':
            return [i for i in range(9) if self.grille[i] == 0]
        else:
            moves = []
            for i in range(9):
                if self.grille[i] == self.joueur_actuel:
                    for j in range(9):
                        if self.grille[j] == 0:
                            # Vous pouvez, ici, ajouter une contrainte pour limiter aux cases adjacentes
                            moves.append((i, j))
            return moves

    def jouer_coup(self, coup):
        """
        Retourne un nouveau Noeud après application du coup.
        En phase placement, 'coup' est un indice (0-8).
        En phase déplacement, 'coup' est un tuple (origine, destination).
        """
        nouvelle_grille = self.grille[:]  # copie de la grille

        if self.nombre_de_pions(self.joueur_actuel) < 3:
            # Phase de placement
            if nouvelle_grille[coup] == 0:
                nouvelle_grille[coup] = self.joueur_actuel
                # Vérifier si, dans la nouvelle grille, les deux joueurs ont 3 pions
                nouvelle_phase = self.phase
                if nouvelle_grille.count(1) == 3 and nouvelle_grille.count(2) == 3:
                    nouvelle_phase = 'déplacement'
                return Noeud(nouvelle_grille, 3 - self.joueur_actuel, nouvelle_phase)
        else:
            # Phase de déplacement : 'coup' est un tuple (origine, destination)
            origine, destination = coup
            if nouvelle_grille[origine] == self.joueur_actuel and nouvelle_grille[destination] == 0:
                nouvelle_grille[origine] = 0
                nouvelle_grille[destination] = self.joueur_actuel
                return Noeud(nouvelle_grille, 3 - self.joueur_actuel, self.phase)
        return None  # Coup invalide

    def get_successor(self):
        """Génère la liste des nœuds successeurs à partir des coups possibles."""
        successeurs = []
        for coup in self.coups_possibles():
            noeud_suivant = self.jouer_coup(coup)
            if noeud_suivant is not None:
                successeurs.append(noeud_suivant)
        return successeurs

    def evaluer(self):
        """
        Évalue la situation du plateau :
         - 1 si le joueur 1 gagne,
         - -1 si le joueur 2 gagne,
         - 0 sinon.
        """
        if self.est_victoire(1):
            return 1
        elif self.est_victoire(2):
            return -1
        return 0

    def minimax(self, profondeur, alpha, beta, maximiser):
        """
        Algorithme minimax avec élagage alpha-bêta.
        Retourne la valeur d'évaluation du nœud.
        La condition d'arrêt ici est :
          - profondeur == 0
          - ou une victoire est détectée (pour le joueur qui vient de jouer)
        """
        if profondeur == 0 or self.joueur_a_gagne():
            return self.evaluer()

        if maximiser:
            max_val = -float('inf')
            for successeur in self.get_successor():
                val = successeur.minimax(profondeur - 1, alpha, beta, False)
                max_val = max(max_val, val)
                alpha = max(alpha, max_val)
                if beta <= alpha:
                    break  # élagage
            return max_val
        else:
            min_val = float('inf')
            for successeur in self.get_successor():
                val = successeur.minimax(profondeur - 1, alpha, beta, True)
                min_val = min(min_val, val)
                beta = min(beta, min_val)
                if beta <= alpha:
                    break  # élagage
            return min_val

    def meilleur_coup(self, profondeur):
        """
        Parcourt tous les coups possibles et retourne celui qui présente la meilleure valeur
        d'après l'algorithme minimax.
        """
        best_move = None
        best_value = -float('inf')
        for coup in self.coups_possibles():
            successeur = self.jouer_coup(coup)
            if successeur is not None:
                move_value = successeur.minimax(profondeur, -float('inf'), float('inf'), False)
                if move_value > best_value:
                    best_value = move_value
                    best_move = coup
        return best_move

    def afficher(self):
        """Affiche la grille de jeu de manière lisible."""
        symboles = {0: '.', 1: 'X', 2: 'O'}
        for i in range(0, 9, 3):
            print(" ".join(symboles[self.grille[i + j]] for j in range(3)))
        print()


def jeu_contre_ia():
    """
    Fonction principale pour jouer contre l'IA.
    Le joueur 1 est l'IA et le joueur 2 est l'humain.
    """
    noeud = Noeud(joueur_actuel=1, phase='placement')
    while not noeud.joueur_a_gagne():
        noeud.afficher()
        if noeud.joueur_actuel == 1:
            print("L'IA joue :")
            coup = noeud.meilleur_coup(profondeur=3)  # Profondeur choisie (peut être ajustée)
            if coup is None:
                print("Aucun coup possible pour l'IA !")
                break
            print("L'IA a choisi :", coup)
            noeud = noeud.jouer_coup(coup)
        else:
            if noeud.phase == 'placement':
                try:
                    coup = int(input("Entrez la position (0-8) pour placer votre pion : "))
                except ValueError:
                    print("Veuillez entrer un entier valide.")
                    continue
            else:
                try:
                    origine = int(input("Choisissez le pion à déplacer (position 0-8) : "))
                    destination = int(input("Choisissez la destination (position 0-8) : "))
                except ValueError:
                    print("Veuillez entrer des entiers valides.")
                    continue
                coup = (origine, destination)
            nouveau_noeud = noeud.jouer_coup(coup)
            if nouveau_noeud is not None:
                noeud = nouveau_noeud
            else:
                print("Coup invalide, réessayez.")

    noeud.afficher()
    if noeud.est_victoire(1):
        print("Le joueur 1 (IA) a gagné !")
    elif noeud.est_victoire(2):
        print("Le joueur 2 (vous) avez gagné !")
    else:
        print("Match nul.")


if __name__ == "__main__":
    jeu_contre_ia()


. . .
. . .
. . .

L'IA joue :
L'IA a choisi : 0
X . .
. . .
. . .

Entrez la position (0-8) pour placer votre pion : 1
X O .
. . .
. . .

L'IA joue :
L'IA a choisi : 2
X O X
. . .
. . .

Entrez la position (0-8) pour placer votre pion : 4
X O X
. O .
. . .

L'IA joue :
L'IA a choisi : 7
X O X
. O .
. X .

Entrez la position (0-8) pour placer votre pion : 5
X O X
. O O
. X .

L'IA joue :
L'IA a choisi : (0, 3)
. O X
X O O
. X .

Choisissez le pion à déplacer (position 0-8) : 0
Choisissez la destination (position 0-8) : 0
Coup invalide, réessayez.
. O X
X O O
. X .

Choisissez le pion à déplacer (position 0-8) : 1
Choisissez la destination (position 0-8) : 0
O . X
X O O
. X .

L'IA joue :
L'IA a choisi : (2, 8)
O . .
X O O
. X X

Choisissez le pion à déplacer (position 0-8) : 5
Choisissez la destination (position 0-8) : 6
O . .
X O .
O X X

L'IA joue :
L'IA a choisi : (7, 2)
O . X
X O .
O . X



KeyboardInterrupt: Interrupted by user